# KNN - (k-Nearest Neighbor)

In [1]:
import numpy as np
import operator

<br>Abaixo segue a implementação do KNN, um algoritmo de machine learning, aprendizagem supervisionada, que classifica um ponto dado através de uma certa quantidade K de pontos próximos a ele (usando distância euclidiana). Um ponto é classificado com base nos tipos de dados que estão próximos a ele.

<br>

![title](images/distance.jpg)
<br>
***Vantagens:***
    * Alta precisão,
    * Insensivel a outliers.

***Desvantagens:***
    * Computacionalmente caro (Lazy learning),
    * Requer muita memória.


## Caso 1: Qualidade de vinho

<justify>O objetivo desse experimento é determinar quais propriedades físico-químicas tornam um vinho 'bom'!

O objetivo desse experimento é coletar dados de um dataset[1] que possuem registros das propriedades físico-químicas de diversos vinhos tintos (como a quantidade ácido fixo, ácido volátil, cloretos, álcool e ph) seguindo de uma nota de 0 a 10 que indica a qualidade do vinho.

![title](images/tabela.PNG)
<br>
A função abaixo lê um arquivo (nesse caso, o winequality-red.csv) e salva os atributos preditores (as propriedades de cada vinho) em uma matriz e os atributos de classe (qualidade) em um array. No dataset, como o atributo "quality" está definido entre 0 e 10, os vinhos com nota maior ou igual a 7 receberam o rótulo de "good" (bom) e os outros receberam "bad".



In [2]:
def readFile(filename):
    file = open(filename, 'r')
    predictors = []# Previsores
    classes = []
    next(file)   # Salta a primeira linha
    for line in file:
        line = line.strip() # remove os \n
        line = line.split(',')
        predictors.append(line[:11])

        # Bom ou ruim
        if(int(line[-1]) >= 7):
            classes.append('good')
        else:
            classes.append('bad')
    predictors = np.array(predictors, dtype =np.float)
    file.close()
    return predictors, classes

In [3]:
predictors, labels = readFile('data/winequality-red.csv')

In [4]:
predictors # Propriedades dos vinhos (dados preditores)

array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
       [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
       [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]])

In [5]:
labels[:20]    # Qualidade (rótulo)

['bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'good',
 'good',
 'bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'good',
 'bad',
 'bad',
 'bad']

#### Normalização

Para que todas as variáveis fiquem na mesma ordem de grandeza, a matriz _predictors_ precisa ser normalizada (colocando as variáveis entre 0 e 1). Fórmula:

![normalizacao](images/normalizacao.png)

In [6]:
def normalize(dataset):
    minValues = dataset.min(axis = 0) # Pega os minimos das colunas
    maxValues = dataset.max(axis = 0)
    #Redimensionando os min e max valores
    minValues = np.tile(minValues, (dataset.shape[0], 1))
    maxValues = np.tile(maxValues, (dataset.shape[0], 1))
    # Fazendo a normalização
    normalizedData = np.zeros(np.shape(dataset))
    normalizedData = (dataset - minValues)/(maxValues - minValues)

    return normalizedData


In [7]:
normalizedMatrix = normalize(predictors)

In [8]:
normalizedMatrix    # Dados normalizados

array([[0.24778761, 0.39726027, 0.        , ..., 0.60629921, 0.13772455,
        0.15384615],
       [0.28318584, 0.52054795, 0.        , ..., 0.36220472, 0.20958084,
        0.21538462],
       [0.28318584, 0.43835616, 0.04      , ..., 0.40944882, 0.19161677,
        0.21538462],
       ...,
       [0.15044248, 0.26712329, 0.13      , ..., 0.53543307, 0.25149701,
        0.4       ],
       [0.11504425, 0.35958904, 0.12      , ..., 0.65354331, 0.22754491,
        0.27692308],
       [0.12389381, 0.13013699, 0.47      , ..., 0.51181102, 0.19760479,
        0.4       ]])

#### Classificador

A função abaixo recebe o valor X que será classificado, a matriz normalizada, os labels e um número inteiro K (Que indica a quantidade de vizinhos mais próximos do X).
<br><br>
Será calculada a distancia euclidiana entre X e os outros valores do dataset, depois será criado um dicionário (Onde os labels são a chave e o valor é a quantidade ) para realizar a "votação" entre os K mais próximos do X. O resultado da classificação será a label que possuir a maior quantidade de votos. 


In [9]:
def classify(X, dataset, labels, k):
    sizeD = dataset.shape[0]
    mEuclideanDistance = np.tile(X, (sizeD, 1)) - dataset
    mEuclideanDistance = mEuclideanDistance**2
    mEuclideanDistance = mEuclideanDistance.sum(axis=1)
    mEuclideanDistance = mEuclideanDistance**0.5
    # Votacao
    ordered = mEuclideanDistance.argsort()                 # Indices ordenados
    voteLabels = {}                                        # Dicionario para realizar a votacao
    for i in range(k):
        label = labels[ordered[i]]
        voteLabels[label] = voteLabels.get(label, 0) + 1   # Adiciona 1 no label correspondente

    voteLabels = sorted(voteLabels.items(), key=operator.itemgetter(1),
                           reverse=True)                   #  Decompõe o dicionario em uma lista de tuplas e "ordena" do maior ao menor
    return voteLabels[0][0]

In [10]:
# Testando a função para classificar oprimeiro elemento
classify(normalizedMatrix[0,:], normalizedMatrix, labels, 3) 

'bad'

#### Testando o classificador

A função abaixo é independente e serve para testar a eficiência/acurácia do classificador (Mandando classificar valores e comparando com a reposta esperada). A variável percentTrain define a porcentagem de dados que serão utilizados para o teste, esses dados serão classificiados com base no resto dos valores do dataset. 


A função também calcula a taxa de erro, que é dada pela soma dos erros do classificador (quando a qualidade do vinho retornada por ele é diferente da qualidade esperada) divido pelo número de testes.

In [11]:
def testeClassificador():
    percentTrain = 0.20 # 20%
    dados, labels = readFile('data/winequality-red.csv')
    normMatriz = normalize(dados)
    m = normMatriz.shape[0] # Tamanho da matriz
    numTeste = int(m*percentTrain)   #Numeros de dados de teste 
    errorCount = 0.0

    for i in range(numTeste):
        resultadoClassificador = classify(normMatriz[i, :], normMatriz[numTeste:m, :],
                                           labels[numTeste:m], 3)
        print("O Classificador retornou: %s, o valor esperado era: %s" % (resultadoClassificador, labels[i]))
        if(resultadoClassificador != labels[i]):
            errorCount += 1.0

    print("Taxa de erro: %f" % (errorCount/float(numTeste)))

In [12]:
testeClassificador()

O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: good
O Classificador retornou: bad, o valor esperado era: good
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: good
O Classificador retornou: ba

O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, o valor esperado era: bad
O Classificador retornou: bad, 

#### Conclusão

A taxa de erro obtida foi <b>0.0752</b>, o que siginifica que o classificador possui cerca de <b>99.93%</b> de precisão.

## Caso 2: Reconhecimento de caracteres manuscritos

O objetivo é classificar digitos manuscritos (de 0 a 9).
<br>
Os dígitos já foram processados e possuem o mesmo tamanho e cor. Além de estarem todos em preto e branco. 32x32.

In [13]:
from os import listdir

#### Ler arquivo de imagem
A função abaixo lê uma imagem (já pré-processada) de dimensões 32x32 e salva em um vetor 1x1024

In [14]:
def imgToVector(filename):
    returnVector = np.zeros((1, 1024))
    file = open(filename)
    contReturnVector = 0        # Ir adicionando no returnVector
    for i in range(32):
        lineStr = file.readline()
        for j in range(32):
            returnVector[0, contReturnVector] = int(lineStr[j])
            contReturnVector = contReturnVector +1
    return returnVector

#### Teste do classificador para imagens
<br>
A função abaixo testa o classificador com imagens, ela acessa 2 diretórios: O trainingDigits e o testDigits que contém imagens pré-processadas de dígitos manuscritos, o nome de cada arquivo é algo como 10_32.txt, onde o 10 é o número que está "escrito" no arquivo e o 32 significa que é a 32ª ocorrência do dígito 10 

In [17]:
def handwritingTest():
    hwLabels = []
    trainingList = listdir('data/trainingDigits')    # Lista com o nome dos arquivos desse diretorio
    m = len(trainingList)
    trainingMat = np.zeros((m, 1024))

    # Guardando os dados de treinamento
    for i in range(m):
        fileNameStr = trainingList[i]
        fileStr = fileNameStr.split('.')[0]     # Pega apenas o nome do arquivo
        label = int(fileStr.split('_')[0])      # Pega o numero correspondente do arquivo
        hwLabels.append(label)
        trainingMat[i, :] = imgToVector('data/trainingDigits/%s' % fileNameStr)

    testList = listdir('data/testDigits')
    errorCount = 0.0
    mTest = len(testList)

    # Comparando os dados de teste com os de treinamento
    for i in range(mTest):
        fileNameStr = testList[i]
        fileStr = fileNameStr.split('.')[0]             # Nome do arquivo (sem .txt)
        Expectedvalue = int(fileNameStr.split('_')[0])    # Número correspondente (esperado)
        vectorForTest = imgToVector('data/testDigits/%s' % fileNameStr)
        result = classify(vectorForTest, trainingMat, hwLabels, 3)
        print('O classificador retornou: %d, valor esperado: %d' % (result, Expectedvalue))

        if(result != Expectedvalue):
            errorCount += 1.0
    print("\nNumero de erros é: %d\n" % errorCount)
    print("\nTaxa de erro é: %f\n" % (errorCount/float(mTest)))

In [18]:
handwritingTest()

O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificador retornou: 0, valor esperado: 0
O classificad

O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 1, valor esperado: 1
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificador retornou: 2, valor esperado: 2
O classificad

O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 3, valor esperado: 3
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificador retornou: 4, valor esperado: 4
O classificad

O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificador retornou: 5, valor esperado: 5
O classificad

O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificador retornou: 7, valor esperado: 7
O classificad

O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificador retornou: 9, valor esperado: 9
O classificad

[1] P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.